# Validation 2.0
### Trying to validation in a different way
##### It seems like we need to validate using **1** user's data. I.e., if we have vol1.csv, vol2.csv, ..., vol15.csv, 23 would train and test the model (for validation step) with only **one** of those files.

##### To me, this doesn't really make sense. If we train a model based off of one user, it will only have seen what that user's data looks like. It will not know what an impostor's data looks like, so it will not know how to classify an impostor.

##### I'm going to try to train with 15% of a single correct user's data.

#### Now realizing, this might be something we want to do on the AIMUX computers since we need to concatanate ALL of the users' data into one dataframe to train the model.

In [21]:
# Straight from chatGPT

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

%matplotlib inline

In [22]:
df = pd.read_csv('../processed-feature-data/all-users-processed-data.csv')
df.tail()

,Unnamed: 0.1,Unnamed: 0,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,...,X_Speed,X_Acceleration,Y_Speed,Y_Acceleration,Speed,Acceleration,Jerk,Ang_V,Path_Tangent,User
768982,38896,38896,1.602808,0.864415,-1.228989,0.096399,-4.558562,-0.871544,-0.007996,4.46888,...,-0.002721,-0.000471,0.007281,-0.000510,-0.182588,-0.008069,-0.002618,0.015651,0.883067,9
768983,38897,38897,1.602841,0.864415,-1.227315,0.096399,-4.558562,-0.871544,-0.007996,4.46888,...,-0.002721,-0.000471,0.006954,-0.000891,-0.182916,-0.008769,-0.003667,0.015651,0.883067,9
768984,38898,38898,1.602902,0.864415,-1.225641,0.096399,-4.558562,-0.871544,-0.007996,4.46888,...,-0.002721,-0.000471,0.004049,-0.001568,-0.185831,-0.010016,-0.003640,0.015651,0.883067,9
768985,38899,38899,1.603026,0.873023,-1.225641,0.096399,-4.558562,-0.871544,-0.007996,4.46888,...,0.006702,0.000964,0.000837,-0.001180,-0.187450,-0.008867,-0.003072,-0.197229,-0.033655,9
768986,38900,38900,1.603241,0.873023,-1.225641,0.096399,-4.558562,1.147389,-0.007996,4.46888,...,-0.002721,-0.001296,0.000837,-0.000705,-0.189055,-0.008679,-0.003234,0.015651,-0.033655,9


#### For this example, let's set user 1 to be the correct user

In [23]:
# Convert any non user 1 values to 0
df.User.replace([1, range(2,16)], [1,0], inplace=True)
df.head()

,Unnamed: 0.1,Unnamed: 0,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,...,X_Speed,X_Acceleration,Y_Speed,Y_Acceleration,Speed,Acceleration,Jerk,Ang_V,Path_Tangent,User
0,0,0,-1.000000,-0.150040,0.595371,0.037739,-1.0,-0.436267,0.037233,-0.263191,...,-0.003502,-0.001640,0.002293,0.000659,-0.027778,-0.000026,-0.000554,0.052891,0.726290,1
1,1,1,-0.999984,-0.154490,0.599727,0.037739,-1.0,-0.436267,0.037233,-0.263191,...,-0.004146,-0.000366,0.003123,0.000647,-0.026918,-0.000591,-0.001320,-0.012641,0.691538,1
2,2,2,-0.999968,-0.160053,0.603212,0.037739,-1.0,-0.436267,0.037233,-0.263191,...,-0.005439,-0.000807,0.002734,0.000080,-0.026726,-0.001119,-0.001316,0.035340,0.761041,1
3,3,3,-0.999952,-0.165615,0.606698,0.037739,-1.0,-0.436267,0.037233,-0.263191,...,-0.005289,0.000120,0.002673,0.000239,-0.026826,-0.001365,-0.001150,0.002761,0.761041,1
4,4,4,-0.999936,-0.172290,0.610183,0.037739,-1.0,-0.436267,0.037233,-0.263191,...,-0.006333,-0.000629,0.002678,0.000270,-0.026386,-0.000919,-0.000762,0.015192,0.788281,1


#### Here we are going to try dropping the features we don't want

In [24]:
# Drop the unnamed 0.1 and unnamed 0 rows, idk what those are.
df=df.drop(['Unnamed: 0.1'],axis=1)
df=df.drop(['Unnamed: 0'],axis=1)
#df.head()

,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,BTN_TOUCH_HELD,BTN_TOUCH_UP,X_Speed,X_Acceleration,Y_Speed,Y_Acceleration,Speed,Acceleration,Jerk,Ang_V,Path_Tangent,User
0,-1.000000,-0.150040,0.595371,0.037739,-1.0,-0.436267,0.037233,-0.263191,0.022278,-0.009902,-0.003502,-0.001640,0.002293,0.000659,-0.027778,-0.000026,-0.000554,0.052891,0.726290,1
1,-0.999984,-0.154490,0.599727,0.037739,-1.0,-0.436267,0.037233,-0.263191,0.022278,-0.009902,-0.004146,-0.000366,0.003123,0.000647,-0.026918,-0.000591,-0.001320,-0.012641,0.691538,1
2,-0.999968,-0.160053,0.603212,0.037739,-1.0,-0.436267,0.037233,-0.263191,0.022278,-0.009902,-0.005439,-0.000807,0.002734,0.000080,-0.026726,-0.001119,-0.001316,0.035340,0.761041,1
3,-0.999952,-0.165615,0.606698,0.037739,-1.0,-0.436267,0.037233,-0.263191,0.022278,-0.009902,-0.005289,0.000120,0.002673,0.000239,-0.026826,-0.001365,-0.001150,0.002761,0.761041,1
4,-0.999936,-0.172290,0.610183,0.037739,-1.0,-0.436267,0.037233,-0.263191,0.022278,-0.009902,-0.006333,-0.000629,0.002678,0.000270,-0.026386,-0.000919,-0.000762,0.015192,0.788281,1


In [5]:
df.iloc[350000]

Unnamed: 0.1      34700.000000
Unnamed: 0        34700.000000
Timestamp             1.206717
X                    -0.085848
Y                     0.309673
TOUCH_MAJOR           0.104544
TOUCH_MINOR           0.283651
TRACKING_ID          -0.869541
PRESSURE             -0.137079
FINGER               -0.404049
BTN_TOUCH_HELD        0.148617
BTN_TOUCH_UP         -0.099618
X_Speed              -0.031644
X_Acceleration       -0.037911
Y_Speed               0.000247
Y_Acceleration        0.001978
Speed                -0.222486
Acceleration         -0.011668
Jerk                 -0.010922
Ang_V                -1.020470
Path_Tangent         -1.457683
User                  0.000000
Name: 350000, dtype: float64

In [25]:
# Generate synthetic data
X, y = df.drop('User', axis=1), df['User']

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Train a Random Forest Classifier on the training set
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Perform validation on the validation set
y_val_pred = clf.predict(X_val)

# Calculate the accuracy on the validation set
validation_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Validation Accuracy: {validation_accuracy}")

Validation Accuracy: 1.0


In [26]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_val,y_val_pred))
print('\n')
print(classification_report(y_val,y_val_pred))

[[106499      0]
 [     0   8849]]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    106499
           1       1.00      1.00      1.00      8849

    accuracy                           1.00    115348
   macro avg       1.00      1.00      1.00    115348
weighted avg       1.00      1.00      1.00    115348

